In [ ]:
from pathlib import Path

root_dir = Path("data", "adaptation")
static_path = root_dir.joinpath("static")
hazard_path =static_path.joinpath("hazard")
network_path = static_path.joinpath("network")
output_path=root_dir.joinpath("output")
tif_files = list(hazard_path.glob("*.tif"))

#calling to the cost file : this should be an intermediate output from previous calculations
input_data = root_dir.joinpath("data")

In [ ]:
from ra2ce.ra2ce_handler import Ra2ceHandler
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.network.network_config_data.network_config_data import (NetworkConfigData,NetworkSection,HazardSection,)
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.network_config_data import (NetworkSection)
from ra2ce.analysis.damages.damages import AnalysisSectionDamages
from ra2ce.analysis.analysis_config_data.analysis_config_data import (AnalysisConfigData,ProjectSection,AnalysisSectionBase,AnalysisSectionLosses)
#Add enums

_network_section = NetworkSection(
    source= SourceEnum.SHAPEFILE,       #Used to specify the shapefile name of the (road) network to do the analysis with, when creating a network from a shapefile.
    primary_file = [network_path.joinpath("network.shp")], #specify in the RA2CE folder setup where the network is locates
    file_id = "ID",
    link_type_column='highway',
    save_gpkg=True
)

_hazard = HazardSection(
    hazard_map=[Path(file) for file in hazard_path.glob("*.tif")],
    hazard_field_name= ['waterdepth'],
    aggregate_wl = AggregateWlEnum.MAX,
    hazard_crs = 'EPSG:4326' 
)

#pass the specified sections as arguments for configuration
_network_config_data = NetworkConfigData(
    root_path=root_dir,
    static_path=static_path,
    output_path=output_path,
    network=_network_section,
    hazard=_hazard
    )

In [ ]:
#specify the parameters for the damage analysis 
damages_analysis = [AnalysisSectionDamages( 
  name = 'RP_damages_HZ', 
  analysis= AnalysisDamagesEnum.DAMAGES, 
  damage_curve = DamageCurveEnum.MAN,  
  input_path = root_dir.joinpath("input_data")                    
  save_csv=True,
  save_gpkg=True,
  hazard_severity_damage_fraction = input_analysis_path.joinpath("hazard_severity_damage_fraction.csv")
  max_damage_road_types = input_analysis_path.joinpath("max_damage_road_types.csv")

  )]

losses_analysis = [AnalysisSectionLosses( 
  name = 'EAL_Multi_link_losses', 
  analysis= AnalysisLossesEnum.MULTI_LINK_LOSSES, 
  event_type = EventTypeEnum.RETURN_PERIOD, 
  weighing = WeighingEnum.TIME,   
  threshold = 0.0,   
  production_loss_per_capita_per_hour =42, 
  traffic_period= TrafficPeriodEnum.DAY, 
  trip_purposes= [TripPurposeEnum.BUSINESS,TripPurposeEnum.COMMUTE,TripPurposeEnum.FREIGHT,TripPurposeEnum.OTHER],
  resilience_curves_file= input_analysis_path.joinpath("resilience_curve.csv"),
  traffic_intensities_file= input_analysis_path.joinpath("traffic_intensities.csv"),
  values_of_time_file= input_analysis_path.joinpath("values_of_time.csv"),

  #save your outputs as both csv and gpkg in the output folder
  save_csv=True,
  save_gpkg=True
  )]

adaptationoption0 = [AdaptationOption(
    id = 0,
    name = 'No adaptation',
)]

adaptationoption1 = [AdaptationOption(
    id = 0,
    name = 'First Adaptation',
    construction_cost = 1000,
    maintenance_interval = 5,
    maintenance_cost = 100,
)]

adaptationoption2 = [AdaptationOption(
    id = 0,
    name = 'Second Adaptation',
    construction_cost = 1000,
    maintenance_interval = 5,
    maintenance_cost = 100,
)]


adaptation_analysis = [AnalysisSectionAdaptation(
    adaptation_options = [adaptationoption1,adaptationoption2],
    damage_analysis = damages_analysis,
    losses_analysis = losses_analysis,
    No_adaptation_option = adaptationoption0,
    discount_rate = 0.05,
    time_horizon = 10,
    vat = 0.21,
    climate_factor = 0.2,
    initial_frequency = 0.2,
    save_csv = True,
    save_gpkg = True    
)]

In [ ]:
analysis_config_data = AnalysisConfigData(
    analyses= adaptation_analysis,
    root_path= root_dir,
    output_path= output_path,
    )

In [ ]:

handler = Ra2ceHandler.from_config(_network_config_data, None)

handler.configure()
handler.run_analysis()